In [1]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# %pip install scipy  --ignore-installed scipy --upgrade
# %pip install pyarrow==0.17.1
# %pip install bayesian-optimization
print("ok")

ok


In [2]:
#!L
# if str(Path.home()) == "/root":  # DATASPHERE
import sys
import gc
from pathlib import Path
sys.path.append(str(Path.cwd()/'riiidNew'))
from pathlib import Path
import numpy as np
import pandas as pd
# import pyarrow
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
from preprocess import preprocess_train_data
from catboost_bayesian_search import bayesian_catboost_search, bayesian_catboost_searchCV, param_adjust_dtypes
import config

In [3]:
# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')


if str(Path.home()) == "/home/sergey":   # Home computer
    kaggle_path = Path.cwd()/"kaggle_tmp/"
    questions_df = pd.read_csv('/mnt/data30G/2020riiid/questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train_1e5.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    # train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid.feather", columns=dtypes.keys())

elif str(Path.home()) == "/root":   # DATASPHERE

    path = Path.cwd()/"riiidNew"/"data1"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_pickle(path/'cv2_train.pickle.zip').astype(config.dtypes, errors="ignore")
    # test_df = pd.read_pickle(path/"cv2_valid.pickle.zip").astype(config.dtypes, errors="ignore")

elif str(Path.home()) == "/home/riiid":   # Virtual machine ycloud

    path = Path.cwd()/"data"
    kaggle_path = Path.cwd()/"kaggle"
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    # train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
    # test_df = pd.read_feather(path/"cv2_test_1e4.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"cv2_train_1e5.feather", columns=config.dtypes.keys())
    # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())
#     train_df = train_df.iloc[:100]

#print(f"train_df shape = {train_df.shape}")

In [4]:
print(config.prior_params)

{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 5000, 'learning_rate': 0.1, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 0.5, 'depth': 10, 'max_leaves': 10, 'border_count': 128, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50}


In [5]:
# # Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
# # train_df = pd.read_feather(path/"train_1e5.feather", columns=config.dtypes.keys())

# # Preprocess for Bayesian CV
# train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
# # test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, config.target, config.dtypes)
# print("Preprocess for CV ok")


# # Training and validating data
# train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
# val_set = Pool(test_df[config.features], label = test_df[config.target])

# print("optimizer_maxCV ..")
# optimizer_max = bayesian_catboost_searchCV(
#     train_set,
#     prior_params=config.prior_params,
#     pds=config.pds, pds_dtypes=config.pds_dtypes,
#     init_points=5, n_iter=7, verbose=True,
# )

In [6]:
horizon = 6
model_name = "model18"

In [7]:
%%time
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
train_df = pd.read_feather(path/"train_5e6.feather", columns=config.dtypes.keys())
# train_df = train_df.iloc[:100]


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


optimizer_max = bayesian_catboost_search(
    train_set, val_set,
    prior_params=config.prior_params,
    pds=config.pds, pds_dtypes=config.pds_dtypes,
    init_points=5, n_iter=37, verbose=True,
)

print("optimizer_max ..")
print(optimizer_max)

Preprocess for model fitting ok
|   iter    |  target   | baggin... | border... |   depth   | l2_lea... | max_le... |
-------------------------------------------------------------------------------------
0:	learn: 0.7408607	test: 0.7186951	best: 0.7186951 (0)	total: 40.8ms	remaining: 3m 24s
250:	learn: 0.7584259	test: 0.7351382	best: 0.7351534 (235)	total: 9.45s	remaining: 2m 58s
bestTest = 0.7352619767
bestIteration = 330
Shrink model to first 331 iterations.
0:	learn: 0.7396066	test: 0.7174248	best: 0.7174248 (0)	total: 39ms	remaining: 3m 14s
250:	learn: 0.7570635	test: 0.7352776	best: 0.7352865 (246)	total: 8.9s	remaining: 2m 48s
bestTest = 0.7353949249
bestIteration = 327
Shrink model to first 328 iterations.
|  2        |  0.7354   |  3.336    |  159.4    |  23.16    |  481.8    |  63.68    |
0:	learn: 0.7298889	test: 0.7077613	best: 0.7077613 (0)	total: 21.6ms	remaining: 1m 48s
250:	learn: 0.7527809	test: 0.7334478	best: 0.7334478 (250)	total: 5.02s	remaining: 1m 35s
500:	learn: 

In [8]:
del train_df
gc.collect()

22

In [9]:
# Obtain new train_df and validation test_df
# train_df = pd.read_feather(path/"cv2_train.feather", columns=config.dtypes.keys())
train_df = pd.read_feather(path/"train.feather", columns=config.dtypes.keys())


# Preprocess for model fitting
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, config.target, config.dtypes)
print("Preprocess for model fitting ok")

# Ratio is 6/24 = 25%
test_df = train_df.groupby('user_id').tail(6)
train_df.drop(test_df.index, inplace = True)

# Training and validating data
train_set = Pool(train_df[config.features], label = train_df[config.target], cat_features=config.cat_features)
val_set = Pool(test_df[config.features], label = test_df[config.target], cat_features=config.cat_features)


# val_set = Pool(valid_df[features], label = valid_df[target])
params = param_adjust_dtypes(config.prior_params,
                             config.pds_dtypes,
                             optimizer_max["params"])
params["learning_rate"] = 6e-3
params['iterations'] = 25000

print("params")
print(params)

model = CatBoostClassifier(**params)

# Fitting
model.fit(train_set, eval_set = val_set, use_best_model = True)
print(f"kaggle_path {kaggle_path}")

model.save_model(f"{kaggle_path/'catboost.{model_name}'}")

print("model.get_best_score()")
print(model.get_best_score())

Preprocess for model fitting ok
params
{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': 'AUC:hints=skip_train~false', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 25000, 'learning_rate': 0.004, 'random_seed': 0, 'bootstrap_type': 'Bayesian', 'l2_leaf_reg': 493.86311678448686, 'depth': 8, 'max_leaves': 12, 'border_count': 280, 'verbose': 250, 'od_type': 'Iter', 'od_wait': 50, 'bagging_temperature': 0.6420623284751135}
0:	learn: 0.7328297	test: 0.7130116	best: 0.7130116 (0)	total: 65.1ms	remaining: 27m 7s
250:	learn: 0.7431636	test: 0.7227285	best: 0.7227285 (250)	total: 16s	remaining: 26m 18s
500:	learn: 0.7457674	test: 0.7256117	best: 0.7256117 (500)	total: 32.3s	remaining: 26m 18s
750:	learn: 0.7478069	test: 0.7284585	best: 0.7284585 (750)	total: 48.6s	remaining: 26m 9s
1000:	learn: 0.7491752	test: 0.7305188	best: 0.7305188 (1000)	total: 1m 4s	remaining: 25m 49s
1250:	learn: 0.7499936	test: 0.7317368	best: 0.7317368 (1250)	total: 1m 20s	remaining: 2

In [10]:
# {'learn': {'Logloss': 0.5831425763098211, 'AUC': 0.7542276382446289}, 'validation': {'Logloss': 0.599964516913287, 'AUC': 0.7367952764034271}}

{'learn': {'Logloss': 0.5845721639824528, 'AUC': 0.7526959180831909},
 'validation': {'Logloss': 0.6011943007661212, 'AUC': 0.7355090975761414}}

In [11]:
from pathlib import Path
Path.home()

PosixPath('/home/riiid')

# Inference

In [12]:
#!M
import pickle

user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.{model_name}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)

In [13]:
train_df.head()
del train_df
gc.collect()

88

In [14]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

CPU times: user 3.73 s, sys: 328 ms, total: 4.06 s
Wall time: 3.84 s


In [15]:
%%time

prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[config.target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[config.target] != -1].reset_index(drop = True)

        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[config.target].values

        # for user_id, content_id, answered_correctly in prior_test_df[["user_id", "content_id", target]].values:
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()

    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)

    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('bool')
    user_sum = np.zeros(len(test_df), dtype = np.int32)
    user_count = np.zeros(len(test_df), dtype = np.int32)
    content_sum = np.zeros(len(test_df), dtype = np.int32)
    content_count = np.zeros(len(test_df), dtype = np.int32)

    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        user_sum[i] = user_sum_dict[user_id]
        user_count[i] = user_count_dict[user_id]
        content_sum[i] = content_sum_dict[content_id]
        content_count[i] = content_count_dict[content_id]

    test_df['user_correctness'] = user_sum / user_count
    test_df['content_count'] = content_count
    test_df['content_id'] = content_sum / content_count
    test_df[config.target] = model.predict_proba(test_df[config.features])[:,1]
    set_predict(test_df[['row_id', config.target]])

KeyboardInterrupt: 